<a href="https://colab.research.google.com/github/wxjmn2007/LLM-Evaluation-on-Math-Problems/blob/main/Llama_3_1_8B_Instruct_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# install dependencies
!pip install transformers
!pip install einops
!pip install accelerate


In [3]:
from huggingface_hub import login
login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset
import torch
import re

model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    device_map = "auto"
)

dataset = load_dataset("gsm8k", "main", split = "train")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

In [4]:
messages = [{"role": "user", "content": "What is 1 + 1?"}]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
).to(model.device)


outputs = model.generate(**inputs, max_new_tokens=40)
output_text = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)

print("Model output:\n", output_text)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Model output:
 The answer to 1 + 1 is 2.


In [1]:
dataset = load_dataset("gsm8k", "main", split = "train")

correct = 0
total = 50

def extract_final_answer(text):
    match = re.search(r"####\s*(-?\d+\.?\d*)", text)
    if match:
        return match.group(1).strip()

    numbers = re.findall(r"-?\d+\.?\d*", text)
    return numbers[-1] if numbers else None



for i in range(total):
  question = dataset[i]["question"]
  expected = dataset[i]["answer"]

  messages = [{"role": "user", "content": f"Solve this math problem step by step: \n{question}"}]

  inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt"
  ).to(model.device)

  outputs = model.generate(**inputs, max_new_tokens = 256)
  output_text = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens = True)

  predicted = extract_final_answer(output_text)
  is_correct = predicted == expected.split("####")[-1].strip()

  if is_correct:
    correct += 1

  print(f"\n Example {i + 1} ")
  print("Question: ", question)
  print("Expected: ", expected)
  print("Predicted:", predicted)
  print("Correct?", is_correct)
  print("Model output: \n", output_text)

print()
accuracy_per = correct / total * 100
accuracy_per = round(accuracy_per, 1)

print("Accuracy: ", correct, "/", total, "=", str(accuracy_per) + "%")



NameError: name 'load_dataset' is not defined